<a href="https://colab.research.google.com/github/labrosse1/RAG-pipeline-Llamaindex/blob/Document-retrieval/RAG_pipeline_Llamaindex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

!pip install python-dotenv

import os
from dotenv import load_dotenv

# Charger le fichier .env
load_dotenv('/content/drive/MyDrive/RAG-pipeline-Llamaindex/.env')

# Récupérer les clés
openai_key = os.environ.get('OPENAI_API_KEY')
scraping_key = os.environ.get('SCRAPING_BEE')

In [ ]:
!pip install scrapingbee

In [ ]:
from scrapingbee import ScrapingBeeClient

client = ScrapingBeeClient(api_key=scraping_key)


urls = ["https://docs.llamaindex.ai/en/stable/understanding/rag/",
        "https://docs.llamaindex.ai/en/stable/understanding/loading/loading/",
        "https://docs.llamaindex.ai/en/stable/understanding/indexing/indexing/",
        "https://docs.llamaindex.ai/en/stable/understanding/storing/storing/",
        "https://docs.llamaindex.ai/en/stable/understanding/querying/querying/",
        ];


for i, url in enumerate(urls, start=1):
    response = client.get(url)
    filename = f"scraped_page{i}.html"

    with open(filename, "w", encoding="utf-8") as f:
      f.write(response.text)


      print(f" {url} sauvegardée dans {filename}")

print('Response HTTP Status Code: ', response.status_code)

In [ ]:
!pip install markdownify

import os
from markdownify import markdownify as md

# Liste pour stocker tous les contenus markdown
all_markdown = []

# Parcourir tous les fichiers HTML du dossier
for file in sorted(os.listdir(".")):
    if file.endswith(".html") and file.startswith("scraped_page"):
        print(f"Conversion de {file} en Markdown...")

        # Lire le HTML
        with open(file, "r", encoding="utf-8") as f:
            html_content = f.read()

        # Convertir en Markdown
        markdown_text = md(html_content)

        # Ajouter un titre séparateur basé sur le nom du fichier
        all_markdown.append(f"# Contenu de {file}\n\n{markdown_text}")

# Joindre tous les fichiers avec deux sauts de ligne
final_markdown = "\n\n---\n\n".join(all_markdown)

# Sauvegarder dans un fichier unique
with open("scraped_all.md", "w", encoding="utf-8") as f:
    f.write(final_markdown)

In [ ]:
!pip install llama-index

In [ ]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("./scraped").load_data()

In [ ]:
!pip install llama-index-vector-stores-chroma


In [ ]:
!pip install chromadb

In [ ]:
import chromadb
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-large")


from llama_index.core.node_parser import SentenceSplitter

text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=10)

# global
from llama_index.core import Settings

Settings.text_splitter = text_splitter
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-large")


# initialize client, setting path to save data
db = chromadb.PersistentClient(path="./chroma_db")

# create collection
chroma_collection = db.get_or_create_collection("quickstart")

# assign chroma as the vector_store to the context
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# per-index
index = VectorStoreIndex.from_documents(
    documents, transformations=[text_splitter],show_progress=True, storage_context=storage_context
)

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query(
    "How to use the query engine"
)
print(response)
